<a href="https://colab.research.google.com/github/sims-smis/IPL_score_winner_prediction/blob/main/IPL_Winner_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as p
import numpy as np

In [2]:
first_dataset = pd.read_csv("IPL_Ball_by_Ball_2008_2022.csv")
second_dataset = pd.read_csv("IPL_Matches_2008_2022.csv")

first_dataset.head()

In [2]:
second_dataset.head()

In [3]:
first_dataset.columns

In [4]:
total_runs = first_dataset.groupby(['ID','innings']).sum()['total_run'].add(1).reset_index()
total_runs

In [5]:
total_runs = total_runs[total_runs['innings']==1]
match_df = second_dataset.merge(total_runs[['ID','total_run']],left_on='ID',right_on='ID')
match_df

In [6]:
match_df['Team1'].value_counts()

In [7]:
match_df['Team1'] = match_df['Team1'].replace('Delhi Daredevils','Delhi Capitals')
match_df['Team2'] = match_df['Team2'].replace('Delhi Daredevils','Delhi Capitals')

match_df['Team1'] = match_df['Team1'].replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['Team2'] = match_df['Team2'].replace('Deccan Chargers','Sunrisers Hyderabad')

match_df['Team1'] = match_df['Team1'].replace('Kings XI Punjab','Punjab Kings')
match_df['Team2'] = match_df['Team2'].replace('Kings XI Punjab','Punjab Kings')

In [8]:
match_df['Team1'].value_counts()

In [9]:
teams = ['Kolkata Knight Riders',
         'Chennai Super Kings',
         'Punjab Kings',
         'Rajasthan Royals',
         'Mumbai Indians',
         'Delhi Capitals',
         'Royal Challengers Bangalore',
         'Sunrisers Hyderabad',
         'Lucknow Super Giants',
         'Gujarat Titans'
         ]

In [10]:
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]

In [11]:
match_df['Team1'].value_counts()

In [12]:
match_df.shape

In [13]:
match_df = match_df[['ID','City','WinningTeam','total_run', 'Team1', 'Team2']]
match_df.shape

In [14]:
second_innings = match_df.merge(first_dataset,on='ID')
second_innings

In [15]:
second_innings = second_innings[second_innings['innings']==2]
second_innings.shape

In [16]:
second_innings['Team1'] = second_innings['Team1'].replace('Delhi Daredevils','Delhi Capitals')
second_innings['Team2'] = second_innings['Team2'].replace('Delhi Daredevils','Delhi Capitals')

second_innings['Team1'] = second_innings['Team1'].replace('Deccan Chargers','Sunrisers Hyderabad')
second_innings['Team2'] = second_innings['Team2'].replace('Deccan Chargers','Sunrisers Hyderabad')

second_innings['Team1'] = second_innings['Team1'].replace('Kings XI Punjab','Punjab Kings')
second_innings['Team2'] = second_innings['Team2'].replace('Kings XI Punjab','Punjab Kings')

In [17]:
second_innings['Team1'].value_counts()

In [18]:
second_innings.columns

In [19]:
second_innings.shape

In [20]:
def determine_bowling_team(row):
    if row['BattingTeam'] == row['Team1']:
        return row['Team2']
    else:
        return row['Team1']

# Create the new 'BowlingTeam' column
second_innings['BowlingTeam'] = second_innings.apply(determine_bowling_team, axis=1)

In [21]:
second_innings.tail(103)

In [22]:
second_innings['BattingTeam'] = second_innings['BattingTeam'].replace('Delhi Daredevils','Delhi Capitals')
second_innings['BowlingTeam'] = second_innings['BowlingTeam'].replace('Delhi Daredevils','Delhi Capitals')

second_innings['BattingTeam'] = second_innings['BattingTeam'].replace('Deccan Chargers','Sunrisers Hyderabad')
second_innings['BowlingTeam'] = second_innings['BowlingTeam'].replace('Deccan Chargers','Sunrisers Hyderabad')

second_innings['BattingTeam'] = second_innings['BattingTeam'].replace('Kings XI Punjab','Punjab Kings')
second_innings['BowlingTeam'] = second_innings['BowlingTeam'].replace('Kings XI Punjab','Punjab Kings')

In [23]:
second_innings

In [24]:
second_innings['BowlingTeam'].unique()

In [25]:
second_innings['current_Score'] = second_innings.groupby('ID').cumsum()['total_run_y']
second_innings

In [26]:
second_innings['runs_left'] = second_innings['total_run_x'] - second_innings['current_Score']
second_innings

In [ ]:
second_innings['balls_left'] = 126 - (second_innings['overs']*6 +second_innings['ballnumber'])
second_innings

In [ ]:
second_innings.columns

In [ ]:
# Select the columns of interest
columns_to_check = ['current_Score', 'runs_left', 'balls_left']

# Check how many values in each column are negative
negative_counts = second_innings[columns_to_check].lt(0).sum()

# Display the counts
print("Number of negative values in each column:")
print(negative_counts)

In [ ]:
second_innings['player_out'] = second_innings['player_out'].fillna("0")
second_innings['player_out'] = second_innings['player_out'].apply(lambda x:x if x=="0" else "1")
second_innings['player_out'] = second_innings['player_out'].astype('int')
wickets = second_innings.groupby('ID').cumsum()['player_out']
second_innings['wickets_remaining'] = 10-wickets
second_innings

In [ ]:
second_innings['crr'] = round((second_innings['current_Score']*6)/(120 - second_innings['balls_left']),2)
second_innings['rrr'] = round((second_innings['runs_left']*6)/(second_innings['balls_left']),2)
second_innings

In [ ]:
def winner(team):
    return 1 if team['BattingTeam'] == team['WinningTeam'] else 0

In [ ]:
second_innings['results'] = second_innings.apply(winner,axis=1)
second_innings

In [ ]:
required_data = second_innings[['BattingTeam','BowlingTeam','City','runs_left','balls_left','wickets_remaining','total_run_x','crr','rrr','results']]
required_data


In [ ]:
required_data = required_data.replace([np.inf, -np.inf], np.nan)
required_data = required_data.dropna()


In [ ]:
required_data = required_data[required_data['crr'] >= 0]

In [ ]:
columns_to_check = ['runs_left', 'balls_left', 'wickets_remaining', 'total_run_x', 'crr', 'rrr', 'results']

required_data = required_data[~(required_data[columns_to_check] < 0).any(axis=1)]

In [ ]:
required_data.head()

In [ ]:
# Assuming your DataFrame is named 'df'
required_data.rename(columns={'BattingTeam': 'batting_team', 'BowlingTeam': 'bowling_team', 'City': 'city'}, inplace=True)
required_data

In [ ]:
numeric = required_data[['runs_left', 'balls_left', 'wickets_remaining', 'total_run_x', 'crr', 'rrr', 'results']]
has_infinite_values = np.any(np.isinf(numeric))

# Check for negative values in the entire DataFrame
has_negative_values = (numeric < 0).any().any()

# Check for null values in the entire DataFrame
has_null_values = numeric.isnull().any().any()

# Display the results
print(f"Has Infinite Values: {has_infinite_values}")
print(f"Has Negative Values: {has_negative_values}")
print(f"Has Null Values: {has_null_values}")

In [ ]:
required_data.to_csv("csv2008-22.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split

x = required_data.iloc[:, :-1]
y = required_data.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=100)
x_train.head()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
],
remainder='passthrough')
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])


pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
pipe.predict_proba(x_test)[0]
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

ra_pipe = Pipeline([
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

ra_pipe.fit(x_train,y_train)

ra_y_pred = ra_pipe.predict(x_test)

accuracy_score(y_test,ra_y_pred)

In [ ]:
import pickle
pickle.dump(ra_pipe,open('ra_pipe.pkl','wb'))
data = pd.read_csv('csv2023.csv')
data.rename(columns={'max_total_run_x': 'total_run_x'}, inplace=True)
data.head(2)

df = pd.concat([required_data, data], ignore_index=True)
df

In [ ]:
df.to_csv('dataset.csv')

In [ ]:
x = df.iloc[:, :-1]
y = df.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=100)
x_train.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

ra_pipe = Pipeline([
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

ra_pipe.fit(x_train,y_train)

ra_y_pred = ra_pipe.predict(x_test)

accuracy_score(y_test,ra_y_pred)

In [ ]:
import pickle
pickle.dump(ra_pipe,open('ra_pipe.pkl','wb'))